Notebook for checking datastores

sometimes optimization crashes or gets stuck during evaluating datastores, to check whether it was caused by the weird datastore feel free to use this notebook.

It works as follows:
- specify the optimization folder
- the jupyter notebook will iterate through folders (supposedly each represent a single mozaik simulation)
- Each found datastore is evaluated and targets' scores are calculated

if no error appears during evaluation of datastore the most probable issue is the memory - it is not enough for the number of jobs that are running in parallel. If the error appears rerun the main optimization script on the nodes with more memory

In [ ]:
import os

# Change the working directory to the directory where the script is located
os.chdir('/home/user/mozaik-tools/Optimization')

import psutil
import json
from importlib import reload
from evaluator import define_fitness_calculator
from targets import TargetValue


import numpy

from mozaik.storage.queries import param_filter_query
from math import isclose
from scipy.spatial.distance import cdist

import utils
reload(utils)
from utils import *

process = psutil.Process()

In [ ]:
# Set the paths for the optimization folder and configuration file

optimization_folder = '/home/user/mozaik-models/LSV1M/20250805-141042_Optimization'
config_optimisation = '/home/user/mozaik-models/LSV1M/param_optim/config_optimisation'

absolute_path = True


In [3]:
with open(config_optimisation) as f:
    opt_config = json.load(f)
fitness_calculator = define_fitness_calculator(opt_config)

In [ ]:
weird_ds = []

for data_store in get_data_stores(optimization_folder, absolute_path=absolute_path):
    print()
    print(data_store.parameters['root_directory'])
    print(f"Memory usage before: {process.memory_info().rss / 1024**3}Gb")
    try:
        scores = fitness_calculator.calculate_scores(data_store)
    except IndexError:
        print("Issue with indexing")
        weird_ds.append(data_store)
        scores = {}
    print(f"Memory usage after: {process.memory_info().rss / 1024**3}Gb")
    print("Scores: ", scores)